In [1]:
# script to Processed the two datasets collected using scrapy and selenium(gdoor.csv and sector.csv)
import pandas as pd
import numpy as np
import re

### Cleaning file containaing company, industry type and sector


In [5]:
file = pd.DataFrame(pd.read_csv('./raw_data/gdoor.csv'))

In [6]:
pd.set_option('max_columns',100)
pd.set_option('max_rows',1000)

In [7]:
f = pd.read_csv('./raw_data/sector.csv', sep='\t', header = None)
f.head()

,0,1,2,3,4,5,6
0,HawkinsPointPartners,"Westford, MA",51 to 200 employees,2012,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals
1,Nylas,"San Francisco, CA",51 to 200 employees,2013,Company - Private,Enterprise Software & Network Solutions,Information Technology
2,MITRE,"Bedford, MA",5001 to 10000 employees,1958,Nonprofit Organization,Federal Agencies,Government
3,Reaktor,"Helsinki, Finland",501 to 1000 employees,2000,Company - Private,Computer Hardware & Software,Information Technology
4,H. Lee Moffitt Cancer Center & Research Institute,"Tampa, FL",5001 to 10000 employees,1984,Hospital,Health Care Services & Hospitals,Health Care


In [8]:
f.columns = ['company', 'address','size','year_founded', 'type', 'industry','sector']

In [9]:
f = f.drop(f.company[f.company.map(lambda x: type(x) is float)].index,0)

f = f.drop(f[f.industry.map(lambda x: type(x) is float)].index,0)

f.sector[f.sector.map(lambda x: type(x) is float)] = 'Unkown'

f.sector[f.sector.str.contains('Unknown')] = f.industry[f.sector.str.contains('Unknown')]

f = f.applymap(lambda x: x.strip())   #or use df.transform(lambda x: x.strip())



### Cleaning file containing list of jobs and respective companies

In [10]:
f2 = pd.read_csv('./raw_data/gdoor.csv')

In [11]:
f2= f2.loc[:,['company','days_posted','job_title','location','max_salary','start_salary','rating']]

f2.loc[:,['company','days_posted','job_title']] \
= f2.loc[:,['company','days_posted','job_title']].applymap(lambda x: x.strip())

f2.company = f2.company.map(lambda x: x.strip()) 

In [12]:
#Extract and normalize posted_days 

def days_posted_extract(x):
    if re.findall('[a-z]+',x)[0] == 'hr':
       val = round(int(re.findall('\d+',x)[0])/24)
    elif re.findall('[a-z]+',x)[0] == 'd':
        val = int(re.findall('\d+',x)[0])
    return val  

In [13]:
#Extract Salary and normalized

def extract_salary(x):
    if type(x) is float:
        val = 0
    elif type(x) is str:
        val = int(re.findall('\d+',x)[0])
    return val

In [14]:
f2['days_posted_c'] = f2['days_posted'].map(days_posted_extract)

salary = f2.loc[:,['start_salary', 'max_salary']].applymap(extract_salary)

f2.rename(columns = {'start_salary': 'start_salary_old' , 'max_salary': 'max_salary_old'}, inplace=True)

f2 = pd.concat([f2,salary],axis ='columns')

#convert rating to whole number by rounding 
f2['rating_c'] = f2.rating.map(lambda x: 0 if pd.isnull(x)   else round(x))

#f2 =f2.reindex(f2.columns.tolist() + ['size','year_founded','type','industry','sector'], axis=1) 

In [15]:
merge_df = pd.merge(f2,f, how='outer', left_on=['company','location'], right_on=['company','address'])

In [16]:
#extract only columns needed for analysis
merge_df = merge_df.loc[:,['company', 'location','job_title', 'days_posted_c', 'start_salary','max_salary','rating_c',\
                           'address','size','year_founded', 'type', 'industry',\
                           'sector']].drop_duplicates().sort_values('company')

In [17]:
#Discard data points with invalid address
merge_df.reset_index(inplace=True)
merge_df.drop(['index'],axis=1, inplace=True)
merge_df = merge_df[merge_df.location.isnull() == False]


In [18]:
#Split address to City and Sate
merge_df['city'] = merge_df.location.map(lambda x: x.split(',')[0] if len(x.split(',')) > 1 else 'Statewide')
merge_df['state']  = merge_df.location.map(lambda x: x.split(',')[1] if len(x.split(',')) > 1 else x)

In [19]:
#Change State names to Abbrev in some data points
merge_df.state[merge_df.state == 'Oregon'] = 'OR'
merge_df.state[merge_df.state == 'Maryland'] = 'MD'
merge_df.state[merge_df.state == 'Virginia'] = 'VA'
merge_df.state[merge_df.state == 'Nevada'] = 'NV'
merge_df.state[merge_df.state == 'Nebraska'] = 'NE'
merge_df.state[merge_df.state == 'Georgia'] = 'GA'
merge_df.state[merge_df.state == 'Wisconsin'] = 'WI'
merge_df.state[merge_df.state == 'New Jersey'] = 'NJ'
merge_df.state[merge_df.state == 'Illinois'] = 'IL'
merge_df.state[merge_df.state == 'New York State'] = 'NY'
merge_df.state[merge_df.state == 'Utah'] = 'UT'
merge_df.state[merge_df.state == 'Remote'] = 'US'
merge_df.state[merge_df.state == 'United States'] = 'US'

C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\patri\Anaconda3\lib\

In [21]:
merge_df.head()

,company,location,job_title,days_posted_c,start_salary,max_salary,rating_c,address,size,year_founded,type,industry,sector,city,state
1,1010data,"New York, NY",Software Data Engineer,12.0,105.0,108.0,3.0,"New York, NY",201 to 500 employees,2000,Company - Private,Computer Hardware & Software,Information Technology,New York,NY
2,1010data,"New York, NY","Sr. Software Development Engineer, Linux",15.0,73.0,147.0,3.0,"New York, NY",201 to 500 employees,2000,Company - Private,Computer Hardware & Software,Information Technology,New York,NY
3,1904labs,"Saint Louis, MO",Data Scientist,19.0,90.0,109.0,5.0,"Saint Louis, MO",51 to 200 employees,2016,Company - Private,IT Services,Information Technology,Saint Louis,MO
5,2U,"New York, NY","Manager, AB Testing and Data Operations",9.0,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,New York,NY
7,2nd.MD,"Houston, TX",Director of Quality Programs & Clinical Analytics,13.0,126.0,158.0,5.0,"Houston, TX",51 to 200 employees,2011,Company - Private,Health Care Services & Hospitals,Health Care,Houston,TX


In [20]:
#Save full processed data
merge_df.to_csv('./clean_data/processed_data.csv', index=False)

In [22]:
#Save a copy of data with only columns needed for analysis
clean_df = merge_df.loc[:,['company','city','state','year_founded','size','rating_c','job_title','days_posted_c',\
             'start_salary','max_salary','industry','sector']]
clean_df.head()

,company,city,state,year_founded,size,rating_c,job_title,days_posted_c,start_salary,max_salary,industry,sector
1,1010data,New York,NY,2000,201 to 500 employees,3.0,Software Data Engineer,12.0,105.0,108.0,Computer Hardware & Software,Information Technology
2,1010data,New York,NY,2000,201 to 500 employees,3.0,"Sr. Software Development Engineer, Linux",15.0,73.0,147.0,Computer Hardware & Software,Information Technology
3,1904labs,Saint Louis,MO,2016,51 to 200 employees,5.0,Data Scientist,19.0,90.0,109.0,IT Services,Information Technology
5,2U,New York,NY,NaN,NaN,4.0,"Manager, AB Testing and Data Operations",9.0,0.0,0.0,NaN,NaN
7,2nd.MD,Houston,TX,2011,51 to 200 employees,5.0,Director of Quality Programs & Clinical Analytics,13.0,126.0,158.0,Health Care Services & Hospitals,Health Care


In [24]:
#Save clean data
clean_df.to_csv('./clean_data/clean_data.csv',index=False)